# Demonstrate FFT filtering


It is also possible to perform manipulations of waveforms in the frequency domain. This has physical applications in low-pass, high-pass, and band-pass filters. Only certain frequencies are allowed in the frequency domain. Once transforming back into the time domain, there are observable effects on the waveform. You can remove high frequencies, low frequencies, or bands of frequencies. 

To demonstrate this, we will look at a different dataset, the number of sunspots in the sun from [the Solar Influences Data Analysis Center](http://www.sidc.be/DATA/yearssn.dat). Sunspots are often accompanied by solar flares, which can affect temperatures on earth. 

However, this dataset is often used by climate change deniers to therefore conclude that humans are not responsible for the warming planet. This dataset does not support the hypothesis quantitatively because the sun spot activity does not correlate with the global temperature anomaly. 

In [ ]:
import matplotlib.pyplot as plt

from fft import fft, fft_power, ifft
import numpy as np
import math

In [ ]:
# Make the plots a bit bigger to see
# NOTE: Must be done in a separate cell
plt.rcParams['figure.dpi'] = 150

## Sunspot Data

These data need to be padded. Once we transform to the Fourier domain, we will see several dominant frequencies. We will investigate what happens when we remove the higher-frequency components. 

If you would like to adjust the amount of filtereding, you can adjust the "maxfreq" variable. 

In [ ]:
from read_ssn import read_ssn
import math

# Read like previous example with CO2 data
x, y = read_ssn('SN_m_tot_V2.0.txt')
y_valid = y >= 0.
y = y[y_valid]

# instead of truncating, pad with values
def pad_data(y, pad_value=0):
    N = len(y)
    log2N_ceil = math.ceil(math.log(N, 2))
    #log2N = math.log(N, 2)
    #next_pow_of_2 = int(log2N) + 1
    if N == 2**log2N_ceil:
        # Input is already a power of 2, nothing to do
        return y
    else:
        # Create new array from inputs + pad values
        print(f"Padding input array of length {N} to length {2**log2N_ceil}")
        y_new = np.ones(2**log2N_ceil) * pad_value
        y_new[:len(y)] = y
        return y_new
y = pad_data(y)
print(f"Input data length = {len(y)}")
                
# Compute the FFT
Y = fft(y)

# filtered the data in the Fourier domain.
# Adjust this to change the frequencies to delete (frequencies are removed from maxfreq to N/2
# and accounts for the Nyquist frequency). 
maxfreq = 50
print(f"Filtering frequencies above k={maxfreq}, f={maxfreq/len(y)}")
print(f"(Equivalently, periods below T={len(y)/maxfreq}) months")
Y_filtered = Y.copy()
Y_filtered[maxfreq:len(Y_filtered)-maxfreq] = 0.0

# Get the absolute value and power for plotting
Yabs = abs(Y)
Ypower = fft_power(Y)
xpower = np.arange(Ypower.size)

Yabs_filtered = abs(Y_filtered)
Ypower_filtered = fft_power(Y_filtered)

# Now go back to the frequency domain. 
# Compare the data before and after filtering. 
y_filtered = ifft(Y_filtered)
yabs_filtered = abs(y_filtered)

fig, axs = plt.subplots(2, 1, figsize=(10, 10))
axs[0].plot(np.arange(len(y)), y, label="Original")
axs[0].plot(np.arange(len(yabs_filtered)), yabs_filtered, label="Filtered")
axs[0].set_xlabel("Index")
axs[0].set_ylabel("Sunspot number")
axs[0].legend()

axs[1].plot(xpower, Ypower, label="Power")
axs[1].plot(xpower, Ypower_filtered, label="Power (filtered)")
#axs[1].plot(np.arange(len(Yabs)), Yabs, label="Magnitude")
axs[1].set_xlabel("Spectral index")
axs[1].set_ylabel("Fourier component")
axs[1].set_yscale("log")
